In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.datasets import mnist
from tensorflow.keras.optimizers import Adam

In [20]:
# Load the dataset
(X_train, y_train), (_, _) = mnist.load_data()

# Configure input
X_train = (X_train.astype(np.float32) - 127.5) / 127.5
X_train = np.expand_dims(X_train, axis=3)
y_train = y_train.reshape(-1, 1)

In [21]:
img_shape = (28, 28, 1)
num_classes = 10
latent_dim = 100

In [22]:
def build_generator():
    model = Sequential()

    model.add(Dense(256, input_dim=latent_dim))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(1024))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(np.prod(img_shape), activation='tanh'))
    model.add(Reshape(img_shape))

    model.summary()

    noise = Input(shape=(latent_dim,))
    label = Input(shape=(1,), dtype='int32')
    label_embedding = Flatten()(Embedding(num_classes, latent_dim)(label))

    model_input = multiply([noise, label_embedding])
    img = model(model_input)

    return Model([noise, label], img)

In [23]:
generator = build_generator()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 256)               25856     
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
batch_normalization_3 (Batch (None, 256)               1024      
_________________________________________________________________
dense_9 (Dense)              (None, 512)               131584    
_________________________________________________________________
leaky_re_lu_7 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
batch_normalization_4 (Batch (None, 512)               2048      
_________________________________________________________________
dense_10 (Dense)             (None, 1024)             

In [24]:
def build_discriminator():
    model = Sequential()

    model.add(Dense(512, input_dim=np.prod(img_shape)))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.4))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.4))
    model.add(Dense(1, activation='sigmoid'))
    model.summary()

    img = Input(shape=img_shape)
    label = Input(shape=(1,), dtype='int32')

    label_embedding = Flatten()(Embedding(num_classes, np.prod(img_shape))(label))
    flat_img = Flatten()(img)

    model_input = multiply([flat_img, label_embedding])

    validity = model(model_input)

    return Model([img, label], validity)

In [25]:
discriminator = build_discriminator()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 512)               401920    
_________________________________________________________________
leaky_re_lu_9 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 512)               262656    
_________________________________________________________________
leaky_re_lu_10 (LeakyReLU)   (None, 512)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_14 (Dense)             (None, 512)               262656    
_________________________________________________________________
leaky_re_lu_11 (LeakyReLU)   (None, 512)              

In [26]:
#-------------------------------------------
# 매개변수 설정
#-------------------------------------------
# 입력 잡음신호의 차원: 100
# 미니배치 크기: 100
# 반복 수: 200
# 학습률 : 0.001
# 생성자의 두 은닉층 크기: 128
# 판별자의 두 은닉층 크기: 128
batch_size = 32 #배치크기
nepochs = 20000 # epochs

In [27]:
noise = Input(shape=(latent_dim,))
label = Input(shape=(1,))

In [28]:
print(noise)
print(label)

Tensor("input_11:0", shape=(None, 100), dtype=float32)
Tensor("input_12:0", shape=(None, 1), dtype=float32)


In [29]:
img = generator([noise,label])
valid = discriminator([img, label])
combined = Model([noise, label], valid)

discriminator.compile(loss=['binary_crossentropy'],
            optimizer=Adam(0.0002, 0.5),
            metrics=['accuracy'])

combined.compile(loss=['binary_crossentropy'],optimizer=Adam(0.0002, 0.5))

In [30]:
# Adversarial ground truths
valids = np.ones((batch_size, 1))
fake = np.zeros((batch_size, 1))

In [36]:
def sample_images(epoch):
    r, c = 2, 5
    n = np.random.normal(0, 1, (r * c, 100))
    sampled_labels = np.arange(0, 10).reshape(-1, 1)

    gen_imgs = generator.predict([n, sampled_labels])

    # Rescale images 0 - 1
    gen_imgs = 0.5 * gen_imgs + 0.5

    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i,j].imshow(gen_imgs[cnt,:,:,0], cmap='gray')
            axs[i,j].set_title("Digit: %d" % sampled_labels[cnt])
            axs[i,j].axis('off')
            cnt += 1
    fig.savefig("images/%d.png" % epoch)
    plt.close()

In [38]:
for epoch in range(nepochs):
    # Train Discriminator
    idx = np.random.randint(0, X_train.shape[0], batch_size)
    imgs, labels = X_train[idx], y_train[idx]
    
    noise = np.random.normal(0, 1, (batch_size, 100))
    
    gen_imgs = generator.predict([noise, labels])
    
    # Train the discriminator
    d_loss_real = discriminator.train_on_batch([imgs, labels], valids)
    d_loss_fake = discriminator.train_on_batch([gen_imgs, labels], fake)
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
    
    # Train Generator
    sampled_labels = np.random.randint(0, 10, batch_size).reshape(-1, 1)
    g_loss = combined.train_on_batch([noise, sampled_labels], valids)
    print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))
    
    # If at save interval => save generated image samples
    if epoch % 200 == 0:
        sample_images(epoch)

0 [D loss: 1.142209, acc.: 51.56%] [G loss: 0.280365]
1 [D loss: 1.094451, acc.: 48.44%] [G loss: 0.208127]
2 [D loss: 1.312890, acc.: 50.00%] [G loss: 0.179410]
3 [D loss: 1.098845, acc.: 50.00%] [G loss: 0.250397]
4 [D loss: 1.002806, acc.: 48.44%] [G loss: 0.254277]
5 [D loss: 1.338810, acc.: 50.00%] [G loss: 0.252241]
6 [D loss: 1.029210, acc.: 50.00%] [G loss: 0.259099]
7 [D loss: 1.203502, acc.: 50.00%] [G loss: 0.191700]
8 [D loss: 1.094432, acc.: 51.56%] [G loss: 0.208097]
9 [D loss: 1.123013, acc.: 50.00%] [G loss: 0.210626]
10 [D loss: 1.074619, acc.: 50.00%] [G loss: 0.282190]
11 [D loss: 1.027118, acc.: 50.00%] [G loss: 0.239497]
12 [D loss: 1.140543, acc.: 50.00%] [G loss: 0.235708]
13 [D loss: 1.112134, acc.: 50.00%] [G loss: 0.250012]
14 [D loss: 1.081076, acc.: 48.44%] [G loss: 0.320792]
15 [D loss: 1.187204, acc.: 48.44%] [G loss: 0.290562]


KeyboardInterrupt: 

In [44]:
class CGAN():
    def __init__(self):
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.num_classes = 10
        self.latent_dim = 100

        optimizer = Adam(0.0002, 0.5)
        
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss=['binary_crossentropy'],
            optimizer=optimizer,
            metrics=['accuracy'])
        
        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise and the target label as input
        # and generates the corresponding digit of that label
        noise = Input(shape=(self.latent_dim,))
        label = Input(shape=(1,))
        img = self.generator([noise, label])

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated image as input and determines validity
        # and the label of that image
        valid = self.discriminator([img, label])

        # The combined model  (stacked generator and discriminator)
        # Trains generator to fool discriminator
        self.combined = Model([noise, label], valid)
        self.combined.compile(loss=['binary_crossentropy'],
            optimizer=optimizer)
        
    def build_generator(self):

        model = Sequential()

        model.add(Dense(256, input_dim=self.latent_dim))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.img_shape), activation='tanh'))
        model.add(Reshape(self.img_shape))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        label = Input(shape=(1,), dtype='int32')
        label_embedding = Flatten()(Embedding(self.num_classes, self.latent_dim)(label))

        model_input = multiply([noise, label_embedding])
        img = model(model_input)

        return Model([noise, label], img)

    def build_discriminator(self):

        model = Sequential()

        model.add(Dense(512, input_dim=np.prod(self.img_shape)))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.4))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.4))
        model.add(Dense(1, activation='sigmoid'))
        model.summary()

        img = Input(shape=self.img_shape)
        label = Input(shape=(1,), dtype='int32')

        label_embedding = Flatten()(Embedding(self.num_classes, np.prod(self.img_shape))(label))
        flat_img = Flatten()(img)

        model_input = multiply([flat_img, label_embedding])

        validity = model(model_input)

        return Model([img, label], validity)
    
    def train(self, epochs, batch_size=128, sample_interval=50):

        # Load the dataset
        (X_train, y_train), (_, _) = mnist.load_data()

        # Configure input
        X_train = (X_train.astype(np.float32) - 127.5) / 127.5
        X_train = np.expand_dims(X_train, axis=3)
        y_train = y_train.reshape(-1, 1)

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random half batch of images
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs, labels = X_train[idx], y_train[idx]

            # Sample noise as generator input
            noise = np.random.normal(0, 1, (batch_size, 100))

            # Generate a half batch of new images
            gen_imgs = self.generator.predict([noise, labels])

            # Train the discriminator
            d_loss_real = self.discriminator.train_on_batch([imgs, labels], valid)
            d_loss_fake = self.discriminator.train_on_batch([gen_imgs, labels], fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  Train Generator
            # ---------------------

            # Condition on labels
            sampled_labels = np.random.randint(0, 10, batch_size).reshape(-1, 1)

            # Train the generator
            g_loss = self.combined.train_on_batch([noise, sampled_labels], valid)

            # Plot the progress
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            # If at save interval => save generated image samples
            if epoch % sample_interval == 0:
                self.sample_images(epoch)
                
    def sample_images(self, epoch):
        r, c = 2, 5
        noise = np.random.normal(0, 1, (r * c, 100))
        sampled_labels = np.arange(0, 10).reshape(-1, 1)

        gen_imgs = self.generator.predict([noise, sampled_labels])

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt,:,:,0], cmap='gray')
                axs[i,j].set_title("Digit: %d" % sampled_labels[cnt])
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("images/%d.png" % epoch)
        plt.close()

In [45]:



if __name__ == '__main__':
    cgan = CGAN()
    cgan.train(epochs=20000, batch_size=32, sample_interval=200)

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_32 (Dense)             (None, 512)               401920    
_________________________________________________________________
leaky_re_lu_24 (LeakyReLU)   (None, 512)               0         
_________________________________________________________________
dense_33 (Dense)             (None, 512)               262656    
_________________________________________________________________
leaky_re_lu_25 (LeakyReLU)   (None, 512)               0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_34 (Dense)             (None, 512)               262656    
_________________________________________________________________
leaky_re_lu_26 (LeakyReLU)   (None, 512)              

89 [D loss: 0.163601, acc.: 92.19%] [G loss: 7.763342]
90 [D loss: 0.074459, acc.: 100.00%] [G loss: 6.970555]
91 [D loss: 0.012732, acc.: 100.00%] [G loss: 6.730814]
92 [D loss: 0.032629, acc.: 98.44%] [G loss: 7.166740]
93 [D loss: 0.062449, acc.: 96.88%] [G loss: 7.576160]
94 [D loss: 0.029166, acc.: 100.00%] [G loss: 7.170207]
95 [D loss: 0.040220, acc.: 100.00%] [G loss: 7.099291]
96 [D loss: 0.057215, acc.: 98.44%] [G loss: 7.448472]
97 [D loss: 0.030188, acc.: 100.00%] [G loss: 6.660264]
98 [D loss: 0.040209, acc.: 96.88%] [G loss: 6.957051]
99 [D loss: 0.030176, acc.: 100.00%] [G loss: 6.965087]
100 [D loss: 0.156144, acc.: 93.75%] [G loss: 7.414396]
101 [D loss: 0.436241, acc.: 87.50%] [G loss: 6.442069]
102 [D loss: 0.179531, acc.: 93.75%] [G loss: 6.913788]
103 [D loss: 0.182540, acc.: 92.19%] [G loss: 7.283479]
104 [D loss: 0.100962, acc.: 96.88%] [G loss: 6.792652]
105 [D loss: 0.208477, acc.: 92.19%] [G loss: 6.289522]
106 [D loss: 0.096073, acc.: 95.31%] [G loss: 7.40732

237 [D loss: 0.373487, acc.: 82.81%] [G loss: 5.026001]
238 [D loss: 0.325790, acc.: 81.25%] [G loss: 4.810604]
239 [D loss: 0.299400, acc.: 84.38%] [G loss: 4.753061]
240 [D loss: 0.201201, acc.: 92.19%] [G loss: 4.459880]
241 [D loss: 0.301972, acc.: 90.62%] [G loss: 5.405654]
242 [D loss: 0.311201, acc.: 85.94%] [G loss: 5.878745]
243 [D loss: 0.205702, acc.: 90.62%] [G loss: 6.350825]
244 [D loss: 0.588780, acc.: 71.88%] [G loss: 4.159742]
245 [D loss: 0.260551, acc.: 89.06%] [G loss: 4.949319]
246 [D loss: 0.274509, acc.: 87.50%] [G loss: 5.161033]
247 [D loss: 0.267427, acc.: 92.19%] [G loss: 4.537479]
248 [D loss: 0.343522, acc.: 84.38%] [G loss: 4.360980]
249 [D loss: 0.381166, acc.: 78.12%] [G loss: 4.212696]
250 [D loss: 0.254416, acc.: 85.94%] [G loss: 5.003431]
251 [D loss: 0.451735, acc.: 82.81%] [G loss: 4.834215]
252 [D loss: 0.388585, acc.: 85.94%] [G loss: 4.663437]
253 [D loss: 0.295843, acc.: 84.38%] [G loss: 4.531687]
254 [D loss: 0.415726, acc.: 82.81%] [G loss: 4.

384 [D loss: 0.378076, acc.: 85.94%] [G loss: 2.292869]
385 [D loss: 0.396188, acc.: 85.94%] [G loss: 1.998598]
386 [D loss: 0.365430, acc.: 81.25%] [G loss: 1.950092]
387 [D loss: 0.420635, acc.: 84.38%] [G loss: 2.767185]
388 [D loss: 0.324791, acc.: 87.50%] [G loss: 2.387043]
389 [D loss: 0.303100, acc.: 90.62%] [G loss: 2.449738]
390 [D loss: 0.355015, acc.: 84.38%] [G loss: 2.206246]
391 [D loss: 0.380128, acc.: 81.25%] [G loss: 1.926203]
392 [D loss: 0.246059, acc.: 92.19%] [G loss: 2.127761]
393 [D loss: 0.404943, acc.: 78.12%] [G loss: 2.731052]
394 [D loss: 0.257034, acc.: 90.62%] [G loss: 3.094388]
395 [D loss: 0.328607, acc.: 84.38%] [G loss: 2.798177]
396 [D loss: 0.417995, acc.: 78.12%] [G loss: 2.362213]
397 [D loss: 0.431531, acc.: 75.00%] [G loss: 2.737167]
398 [D loss: 0.394018, acc.: 87.50%] [G loss: 2.555642]
399 [D loss: 0.571545, acc.: 73.44%] [G loss: 2.357255]
400 [D loss: 0.445492, acc.: 78.12%] [G loss: 2.404502]
401 [D loss: 0.398082, acc.: 84.38%] [G loss: 2.

531 [D loss: 0.531750, acc.: 64.06%] [G loss: 1.920576]
532 [D loss: 0.521048, acc.: 68.75%] [G loss: 1.586869]
533 [D loss: 0.574046, acc.: 67.19%] [G loss: 1.773523]
534 [D loss: 0.508558, acc.: 79.69%] [G loss: 1.690147]
535 [D loss: 0.363132, acc.: 87.50%] [G loss: 1.498092]
536 [D loss: 0.545086, acc.: 73.44%] [G loss: 1.426098]
537 [D loss: 0.438295, acc.: 82.81%] [G loss: 1.833245]
538 [D loss: 0.408167, acc.: 81.25%] [G loss: 1.967962]
539 [D loss: 0.591212, acc.: 71.88%] [G loss: 1.572718]
540 [D loss: 0.521735, acc.: 73.44%] [G loss: 1.504502]
541 [D loss: 0.542462, acc.: 73.44%] [G loss: 1.667371]
542 [D loss: 0.585337, acc.: 68.75%] [G loss: 1.947785]
543 [D loss: 0.481750, acc.: 78.12%] [G loss: 1.705548]
544 [D loss: 0.587267, acc.: 68.75%] [G loss: 1.455230]
545 [D loss: 0.461214, acc.: 79.69%] [G loss: 1.361843]
546 [D loss: 0.618288, acc.: 64.06%] [G loss: 1.526093]
547 [D loss: 0.528280, acc.: 75.00%] [G loss: 1.731191]
548 [D loss: 0.461822, acc.: 73.44%] [G loss: 1.

...

19313 [D loss: 0.700806, acc.: 48.44%] [G loss: 0.775642]
19314 [D loss: 0.648764, acc.: 70.31%] [G loss: 0.772562]
19315 [D loss: 0.694174, acc.: 50.00%] [G loss: 0.758557]
19316 [D loss: 0.669305, acc.: 62.50%] [G loss: 0.724497]
19317 [D loss: 0.673509, acc.: 51.56%] [G loss: 0.766101]
19318 [D loss: 0.659938, acc.: 60.94%] [G loss: 0.751541]
19319 [D loss: 0.665060, acc.: 51.56%] [G loss: 0.746494]
19320 [D loss: 0.643046, acc.: 57.81%] [G loss: 0.759309]
19321 [D loss: 0.694013, acc.: 53.12%] [G loss: 0.779471]
19322 [D loss: 0.674495, acc.: 59.38%] [G loss: 0.803284]
19323 [D loss: 0.696724, acc.: 46.88%] [G loss: 0.778152]
19324 [D loss: 0.655372, acc.: 54.69%] [G loss: 0.751105]
19325 [D loss: 0.689968, acc.: 48.44%] [G loss: 0.749119]
19326 [D loss: 0.719650, acc.: 51.56%] [G loss: 0.785463]
19327 [D loss: 0.657589, acc.: 62.50%] [G loss: 0.776710]
19328 [D loss: 0.686264, acc.: 57.81%] [G loss: 0.801128]
19329 [D loss: 0.633274, acc.: 65.62%] [G loss: 0.778563]
19330 [D loss:

19456 [D loss: 0.707860, acc.: 50.00%] [G loss: 0.784244]
19457 [D loss: 0.657780, acc.: 64.06%] [G loss: 0.852276]
19458 [D loss: 0.700254, acc.: 43.75%] [G loss: 0.793698]
19459 [D loss: 0.697352, acc.: 48.44%] [G loss: 0.751168]
19460 [D loss: 0.681961, acc.: 54.69%] [G loss: 0.772295]
19461 [D loss: 0.649126, acc.: 60.94%] [G loss: 0.836076]
19462 [D loss: 0.678889, acc.: 51.56%] [G loss: 0.767029]
19463 [D loss: 0.674067, acc.: 54.69%] [G loss: 0.779007]
19464 [D loss: 0.664295, acc.: 60.94%] [G loss: 0.781557]
19465 [D loss: 0.687858, acc.: 48.44%] [G loss: 0.793859]
19466 [D loss: 0.667513, acc.: 62.50%] [G loss: 0.799679]
19467 [D loss: 0.668543, acc.: 54.69%] [G loss: 0.762151]
19468 [D loss: 0.685921, acc.: 62.50%] [G loss: 0.776917]
19469 [D loss: 0.644094, acc.: 64.06%] [G loss: 0.738645]
19470 [D loss: 0.688937, acc.: 53.12%] [G loss: 0.790999]
19471 [D loss: 0.709760, acc.: 53.12%] [G loss: 0.734267]
19472 [D loss: 0.705191, acc.: 46.88%] [G loss: 0.801520]
19473 [D loss:

19599 [D loss: 0.675158, acc.: 53.12%] [G loss: 0.781822]
19600 [D loss: 0.708529, acc.: 51.56%] [G loss: 0.780986]
19601 [D loss: 0.689452, acc.: 50.00%] [G loss: 0.711076]
19602 [D loss: 0.726198, acc.: 57.81%] [G loss: 0.816596]
19603 [D loss: 0.650724, acc.: 56.25%] [G loss: 0.829367]
19604 [D loss: 0.678696, acc.: 57.81%] [G loss: 0.837751]
19605 [D loss: 0.688057, acc.: 56.25%] [G loss: 0.827667]
19606 [D loss: 0.674244, acc.: 57.81%] [G loss: 0.773140]
19607 [D loss: 0.693167, acc.: 48.44%] [G loss: 0.761930]
19608 [D loss: 0.674090, acc.: 54.69%] [G loss: 0.806241]
19609 [D loss: 0.659507, acc.: 64.06%] [G loss: 0.803525]
19610 [D loss: 0.674020, acc.: 51.56%] [G loss: 0.796486]
19611 [D loss: 0.728034, acc.: 43.75%] [G loss: 0.833278]
19612 [D loss: 0.670742, acc.: 60.94%] [G loss: 0.782466]
19613 [D loss: 0.657050, acc.: 59.38%] [G loss: 0.818020]
19614 [D loss: 0.697482, acc.: 50.00%] [G loss: 0.830818]
19615 [D loss: 0.660068, acc.: 59.38%] [G loss: 0.794760]
19616 [D loss:

19742 [D loss: 0.676802, acc.: 53.12%] [G loss: 0.725730]
19743 [D loss: 0.673257, acc.: 60.94%] [G loss: 0.733099]
19744 [D loss: 0.669466, acc.: 65.62%] [G loss: 0.764415]
19745 [D loss: 0.713661, acc.: 43.75%] [G loss: 0.769793]
19746 [D loss: 0.653073, acc.: 59.38%] [G loss: 0.748612]
19747 [D loss: 0.717580, acc.: 45.31%] [G loss: 0.779790]
19748 [D loss: 0.702820, acc.: 57.81%] [G loss: 0.738566]
19749 [D loss: 0.693581, acc.: 46.88%] [G loss: 0.724160]
19750 [D loss: 0.655459, acc.: 54.69%] [G loss: 0.781605]
19751 [D loss: 0.667335, acc.: 60.94%] [G loss: 0.790154]
19752 [D loss: 0.701517, acc.: 46.88%] [G loss: 0.830513]
19753 [D loss: 0.643239, acc.: 59.38%] [G loss: 0.774586]
19754 [D loss: 0.687939, acc.: 50.00%] [G loss: 0.827381]
19755 [D loss: 0.635128, acc.: 67.19%] [G loss: 0.764522]
19756 [D loss: 0.649210, acc.: 57.81%] [G loss: 0.782368]
19757 [D loss: 0.648376, acc.: 67.19%] [G loss: 0.787395]
19758 [D loss: 0.665133, acc.: 64.06%] [G loss: 0.803827]
19759 [D loss:

19885 [D loss: 0.653795, acc.: 67.19%] [G loss: 0.806184]
19886 [D loss: 0.671475, acc.: 57.81%] [G loss: 0.821254]
19887 [D loss: 0.680982, acc.: 59.38%] [G loss: 0.756874]
19888 [D loss: 0.676255, acc.: 59.38%] [G loss: 0.766552]
19889 [D loss: 0.630374, acc.: 65.62%] [G loss: 0.792123]
19890 [D loss: 0.706830, acc.: 53.12%] [G loss: 0.710481]
19891 [D loss: 0.702991, acc.: 54.69%] [G loss: 0.769940]
19892 [D loss: 0.713898, acc.: 46.88%] [G loss: 0.794898]
19893 [D loss: 0.626141, acc.: 60.94%] [G loss: 0.764156]
19894 [D loss: 0.666986, acc.: 57.81%] [G loss: 0.772702]
19895 [D loss: 0.659907, acc.: 54.69%] [G loss: 0.803453]
19896 [D loss: 0.727927, acc.: 48.44%] [G loss: 0.794564]
19897 [D loss: 0.682630, acc.: 57.81%] [G loss: 0.811254]
19898 [D loss: 0.651471, acc.: 54.69%] [G loss: 0.813890]
19899 [D loss: 0.692229, acc.: 50.00%] [G loss: 0.780423]
19900 [D loss: 0.653894, acc.: 62.50%] [G loss: 0.780258]
19901 [D loss: 0.653867, acc.: 64.06%] [G loss: 0.810188]
19902 [D loss: